In [7]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-2.4.4-bin-hadoop2.7'

In [8]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import*
from pyspark.sql.functions import*

In [10]:
conf = SparkConf().setAppName('eleflow').setMaster('local').set('spark.driver.host', 'localhost')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [11]:
spark

In [169]:
schema = StructType([
    StructField('mes', StringType()),
    StructField('1', StringType()),
    StructField('regiao', StringType()),
    StructField('estado', StringType()),
    StructField('municipio', StringType()),
    StructField('produto', StringType()),
    StructField('n_postos_pesquisados', StringType()),
    StructField('unidade_medida', StringType()),
    StructField('rev_preco_med', StringType()),
    StructField('2', StringType()),
    StructField('rev_preco_min', StringType()),
    StructField('rev_preco_max', StringType()),
    StructField('3', StringType()),
    StructField('4', StringType()),
    StructField('dist_preco_med', StringType()),
    StructField('5', StringType()),
    StructField('dist_preco_min', StringType()),
    StructField('dist_preco_max', StringType()),
    StructField('6', StringType()),
])

sqlContext = SQLContext(spark.sparkContext)
df = spark.read.csv(path='dataframe.csv', header=True, schema=schema).cache()

In [170]:
df = df.withColumn('mes', 
                to_date(unix_timestamp(col('mes'), 'dd/MM/yyyy').cast('timestamp')))
df = df.filter(year('mes') > 2018)
df = df.withColumn('mes', month('mes'))
    
float_lst = ['',
       'rev_preco_min',
       'rev_preco_med,
       'rev_preco_max',
       'dist_preco_min',
       'dist_preco_med',
       'dist_preco_max',
      ]

for i in float_lst:
    df = df.withColumn(i, regexp_replace(i, ',', '.'))
    df = df.withColumn(i, df[i].cast(FloatType()))
    
df = df.withColumn('n_postos_pesquisados', df.n_postos_pesquisados.cast(IntegerType()))
df = df.fillna(0)
df = df.drop('data_final')
df = df.drop('rev_desvio_padrao')
df = df.drop('rev_margem_media')
df = df.drop('rev_coef_variacao')
df = df.drop('dist_coef_variacao')
df = df.drop('dist_preco_padrao')
df = df.drop('dist_coef_variacao')


In [171]:
df.printSchema()


root
 |-- mes: integer (nullable = true)
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- produto: string (nullable = true)
 |-- n_postos_pesquisados: integer (nullable = true)
 |-- unidade_medida: string (nullable = true)
 |-- rev_preco_medio: float (nullable = false)
 |-- rev_preco_minimo: float (nullable = false)
 |-- rev_preco_maximo: float (nullable = false)
 |-- dist_preco_medio: float (nullable = false)
 |-- dist_preco_minimo: float (nullable = false)
 |-- dist_preco_maximo: float (nullable = false)



In [172]:
df.write.csv('output', header=True)